# TFG: Título del TFG

## Hugo López Álvarez

In [ ]:
import math
import numpy    
import pandas   
import wandb
import torch    
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, log_loss, fbeta_score
from sklearn.model_selection import KFold, StratifiedKFold

## Clases

Definición de la clase DatasetTFG que se usará para entrenar al modelo

In [ ]:
class DatasetTFG(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

Definición de la clase Modelo
- La capa1 transforma la dimensión de entrada a 64 neuronas
- La capa2 pasa de las 64 neuronas a 1 neurona

In [ ]:
class Modelo(nn.Module):
    def __init__(self, input_dim, ventanaOculta):
        super().__init__()
        self.capa1 = nn.Linear(input_dim, ventanaOculta)
        self.capa2 =  nn.Linear(ventanaOculta, 1)
        self.sigprueba = nn.Sigmoid() #Se pasa el volor por una sigmoidea
        
    def forward(self,  X):
        X = self.capa1(X)
        X = self.capa2(X)
        #X = self.sigprueba(X)
        return X

# Funciones

In [ ]:

def split_ip_column(df, ip_column_name):
    
    # Divide la IP en cuatro partes
    ip_parts = df[ip_column_name].str.split('.', expand=True)
    
    # Crea nombres de columnas basados en el nombre original
    new_columns = {
        0: f"{ip_column_name}_part1",
        1: f"{ip_column_name}_part2", 
        2: f"{ip_column_name}_part3",
        3: f"{ip_column_name}_part4"
    }
    
    # Se elimina la columna de ip_column_name
    df = df.drop(columns=[ip_column_name]) 
    
    # Añade las nuevas columnas al DataFrame
    for part, col_name in new_columns.items():
        df[col_name] = pandas.to_numeric(ip_parts[part])  # Convierte a numérico
    
    return df

## Cargar datos

In [ ]:
fileData = pandas.read_csv('../Datasets/modUQ.csv')

### Comprobación de la obtención correcta del csv

In [ ]:
fileData.head()

### Se convierten las columnas no numéricas para poder utilizarlas con pytorch

In [ ]:
#fileData = split_ip_column(fileData, 'IPV4_SRC_ADDR')
#fileData = split_ip_column(fileData, 'IPV4_DST_ADDR')
fileData['Attack'] = LabelEncoder().fit_transform(fileData['Attack'])

### Se comprueba que los datos se han transformado correctamente

In [ ]:
print(fileData.dtypes)

## Se eliminan los datos con valores infinitos

In [ ]:
#print("¿Existen valores infinitos en X?: ", numpy.isinf(fileData.values).any())
fileData = fileData.replace([numpy.inf, -numpy.inf], numpy.nan).dropna()
#print("¿Siguen existiendo valores infinitos en X?: ", numpy.isinf(fileData.values).any())

### Se separan las características (X) de la etiqueta (Y)

In [ ]:
X = fileData.drop(columns=['Label', 'Attack', 'FLOW_START_MILLISECONDS', 'FLOW_END_MILLISECONDS', 'IPV4_SRC_ADDR', 'IPV4_DST_ADDR']).values
Y = fileData['Label'].values

### Se elimina fileData que contiene el csv con los datoa para liberar memoria

In [ ]:
del fileData

## Se separan los datos del entrenamiento de los datos de prueba
El entrenamiento tendrá el 80% de los datos

La prueba tendrá el 20% de los datos

In [ ]:
X_entrena, X_prueba, Y_entrana, Y_prueba = train_test_split(
    X, Y, test_size=0.2, random_state=42,  stratify=Y
)

## Se normalizan los datos

In [ ]:
escalador = MinMaxScaler(feature_range=(0,1))
X_entrena_normalizado = escalador.fit_transform(X_entrena)

### Se convierten los datos a tensores de Pytorch

In [ ]:
X_entrena_tensor = torch.tensor(X_entrena_normalizado, dtype=torch.float32)
Y_entrena_tensor = torch.tensor(Y_entrana, dtype=torch.float32)

## Creación del Dataset personalizado

In [ ]:
dataset_entrena = DatasetTFG(X_entrena_tensor, Y_entrena_tensor)

## Se configura pérdida y optimizador

In [ ]:
#perdida = nn.BCELoss() if len(Y_entrena_tensor.unique()) > 2 else nn.CrossEntropyLoss()
pos_weight = torch.tensor([(len(Y_entrana)-sum(Y_entrana))/sum(Y_entrana)])  # Auto-cálculo
perdida = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

## Selección de GPU si existe

Los resultados han sido peores de lo esperado utilizando la GPU frente a la CPU
![alt text](./imgs/resultadosGPU.png)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Entrenando en: {device}")   

## Definición de hiperparámetros

In [ ]:
batch_size=[2000, 10000, 15000, 20000]
learning_rate=[0.01, 0.001, 0.0001]
epochs=[10, 20, 30]
hidden_factor=[math.ceil(X_entrena_tensor.size(1)/2), X_entrena_tensor.size(1), X_entrena_tensor.size(1)*2] # la mitad de las columnas, el número de columnas y el doble

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## Bucle de entrenamiento o épocas

In [ ]:
for bs in batch_size:
    for lr in learning_rate:
        for hs in hidden_factor:
            for e in epochs:
                # Listas para almacenar métricas de cada fold
                fold_metrics = {
                    'loss': [], 'accuracy': [], 'precision': [], 
                    'recall': [], 'f1': [], 'f2': [], 'roc_auc': [],
                    'specificity': [], 'tp': [], 'fp': [], 'tn': [], 'fn': []
                }
                
                # Validación cruzada
                for fold, (train_idx, val_idx) in enumerate(kf.split(X_entrena_tensor.cpu(), Y_entrena_tensor.cpu())):
                    print(f"\n--- Fold {fold+1} ---")
                    # Configuración del experimento en wandb (por fold)
                    nombreExperimento = f'TFG_BIN_bs({bs})_lr({lr})_hs({hs})_e({e})_fold({fold+1})'
                    wandb.init(
                        project="TFG_BIN_CV_FOLDS",
                        name=nombreExperimento,
                        config={
                            "batch_size": bs,
                            "learning_rate": lr,
                            "hidden_size": hs,
                            "epochs": e,
                            "fold": fold+1,
                            "device": str(device)
                        }
                    )
                    
                    # Divisón train/val para este fold
                    X_train_fold = X_entrena_tensor[train_idx].to(device)
                    Y_train_fold = Y_entrena_tensor[train_idx].to(device)
                    X_val_fold = X_entrena_tensor[val_idx].to(device)
                    Y_val_fold = Y_entrena_tensor[val_idx].to(device)
                    
                    train_data = torch.utils.data.TensorDataset(X_train_fold, Y_train_fold)
                    val_data = torch.utils.data.TensorDataset(X_val_fold, Y_val_fold)

                    train_loader = DataLoader(train_data, batch_size=bs, shuffle=True)
                    val_loader = DataLoader(val_data, batch_size=bs)
                    
                    # Modelo y optimizador
                    modelo = Modelo(input_dim=X_entrena_tensor.shape[1], ventanaOculta=hs).to(device)
                    optimizador = optim.AdamW(modelo.parameters(), lr=lr)
                    perdida = nn.BCEWithLogitsLoss().to(device)
                    
                    # Entrenamiento
                    for epoch in range(e):
                        modelo.train()
                        for batch_X, batch_Y in train_loader:
                            optimizador.zero_grad()
                            salidas = modelo(batch_X)
                            loss = perdida(salidas, batch_Y.unsqueeze(1))
                            loss.backward()
                            optimizador.step()
                    
                    # Evaluación en validation fold
                    modelo.eval()
                    val_preds, val_probs, val_targets = [], [], []
                    val_loss = 0.0
                    
                    with torch.no_grad():
                        for batch_X_val, batch_Y_val in val_loader:
                            salidas_val = modelo(batch_X_val)
                            val_loss += perdida(salidas_val, batch_Y_val.unsqueeze(1)).item()
                            probs = torch.sigmoid(salidas_val)
                            preds = (probs > 0.5).int()
                            val_probs.extend(probs.cpu().numpy())
                            val_preds.extend(preds.cpu().numpy())
                            val_targets.extend(batch_Y_val.cpu().numpy())
                    
                    # Métricas para este fold
                    val_loss /= len(val_loader)
                    tn, fp, fn, tp = confusion_matrix(val_targets, val_preds).ravel()
                    
                    fold_metrics['loss'].append(val_loss)
                    fold_metrics['accuracy'].append(accuracy_score(val_targets, val_preds))
                    fold_metrics['precision'].append(precision_score(val_targets, val_preds, zero_division=0))
                    fold_metrics['recall'].append(recall_score(val_targets, val_preds, zero_division=0))
                    fold_metrics['f1'].append(f1_score(val_targets, val_preds))
                    fold_metrics['f2'].append(fbeta_score(val_targets, val_preds, beta=2))
                    fold_metrics['roc_auc'].append(roc_auc_score(val_targets, val_probs))
                    fold_metrics['specificity'].append(tn / (tn + fp) if (tn + fp) > 0 else 0)
                    fold_metrics['tp'].append(tp)
                    fold_metrics['fp'].append(fp)
                    fold_metrics['tn'].append(tn)
                    fold_metrics['fn'].append(fn)
                    
                    # Log metrics por fold
                    wandb.log({
                        "fold": fold+1,
                        "loss": val_loss,
                        "accuracy": fold_metrics['accuracy'][-1],
                        "precision": fold_metrics['precision'][-1],
                        "recall": fold_metrics['recall'][-1],
                        "f1": fold_metrics['f1'][-1],
                        "f2": fold_metrics['f2'][-1],
                        "roc_auc": fold_metrics['roc_auc'][-1],
                        "specificity": fold_metrics['specificity'][-1],
                        "true_positives": tp,
                        "false_positives": fp,
                        "true_negatives": tn,
                        "false_negatives": fn
                    })
                    
                    wandb.finish()
                
                # Cálculo de métricas promedio (media de los K folds)
                avg_metrics = {k: numpy.mean(v) for k, v in fold_metrics.items()}
                
                # Log de métricas promedio en wandb (experimento resumen)
                wandb.init(
                    project="TFG_BIN_CV_AVG                     ",
                    name=f"AVG_bs({bs})_lr({lr})_hs({hs})_e({e})",
                    config={
                        "batch_size": bs,
                        "learning_rate": lr,
                        "hidden_size": hs,
                        "epochs": e,
                        "device": str(device)
                    }
                )
                
                wandb.log({
                    "avg_loss": avg_metrics['loss'],
                    "avg_accuracy": avg_metrics['accuracy'],
                    "avg_precision": avg_metrics['precision'],
                    "avg_recall": avg_metrics['recall'],
                    "avg_f1": avg_metrics['f1'],
                    "avg_f2": avg_metrics['f2'],
                    "avg_roc_auc": avg_metrics['roc_auc'],
                    "avg_specificity": avg_metrics['specificity'],
                    "avg_tp": avg_metrics['tp'],
                    "avg_fp": avg_metrics['fp'],
                    "avg_tn": avg_metrics['tn'],
                    "avg_fn": avg_metrics['fn']
                })
                
                print(f"\nResultados promedio para bs={bs}, lr={lr}, hs={hs}, e={e}:")
                print(f"  Loss: {avg_metrics['loss']:.4f} | Accuracy: {avg_metrics['accuracy']:.4f} | F1: {avg_metrics['f1']:.4f}")
                
                wandb.finish()

## Se preparan los datos de prueba

In [ ]:
X_prueba_normalizado = escalador.transform(X_prueba)

X_prueba_tensor = torch.tensor(X_prueba_normalizado, dtype=torch.float32)
Y_prueba_tensor = torch.tensor(Y_prueba, dtype=torch.float32)

dataset_prueba = DatasetTFG(X_prueba_tensor, Y_prueba_tensor)


## Se guarda el modelo en un fichero

In [ ]:
torch.save(modelo.state_dict(), 'modeloSinLabelNiAttackNiIPSRC.pth')

### BCEWithLogitsLoss

Época: 1, Pérdida: 0.020396
Época: 2, Pérdida: 0.009877
Época: 3, Pérdida: 0.004355
Época: 4, Pérdida: 0.00114
Época: 5, Pérdida: 0.002756
Época: 6, Pérdida: 0.001659
Época: 7, Pérdida: 0.004422
Época: 8, Pérdida: 0.000846
Época: 9, Pérdida: 0.000706
Época: 10, Pérdida: 0.000514

Pérdida durante la prueba: 0.0075, Exactitud:  95.90%
